In [1]:
import pandas as pd
import os

In [2]:
%pwd

'd:\\DL\\Live project\\DL-Inuron_Assignment_2\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\DL\\Live project\\DL-Inuron_Assignment_2'

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    test_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [18]:
from dl_assignment2.constants import *
from dl_assignment2.constants import *
from dl_assignment2.utils.common import read_yaml, create_directories, save_json
import tensorflow as tf

In [24]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=self.config.evaluation.path_of_model,
            training_data=self.config.evaluation.training_data,
            test_data=Path(self.config.evaluation.test_data),
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [14]:
import tensorflow as tf
import mlflow
import mlflow.keras
from urllib.parse import urlparse

import tensorflow as tf
import time
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

In [26]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    
    def load_cifar10_batch(self,file_path):
        with open(file_path, 'rb') as f:
            data_dict = pickle.load(f, encoding='bytes')
        return data_dict[b'data'], np.array(data_dict[b'labels'])

    def evaluation(self,):
 
        # Load testing data
        X_test, y_test = self.load_cifar10_batch(self.config.test_data)

        X_test = X_test.reshape(-1, 32, 32, 3)

        X_test = X_test.astype('float32') / 255.0

        y_test = to_categorical(y_test, 100)
        

        model = load_model(self.config.path_of_model)
        loss, acc = model.evaluate(X_test, y_test)
        scores = {"loss": loss, "accuracy": acc}
        save_json(path=Path("scores.json"), data=scores)
    



In [27]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()

except Exception as e:
   raise e

[2025-03-04 16:26:11,120: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-04 16:26:11,131: INFO: common: yaml file: params.yaml loaded successfully]


[2025-03-04 16:26:11,137: INFO: common: created directory at: artifacts]
313/313 [==============================] - 11s 31ms/step - loss: 1.7350 - accuracy: 0.3752
[2025-03-04 16:26:24,512: INFO: common: json file saved at: scores.json]


In [74]:
import matplotlib.pyplot as plt
config = ConfigurationManager()
training_config = config.get_training_config()
training = Training(config=training_config)
X_train, X_test, y_train, y_test = training.load_data()

# Load CIFAR-10 class names
with open("artifacts/data_ingestion/cifar-10-batches-py/batches.meta", 'rb') as f:
    meta = pickle.load(f, encoding='bytes')
class_names = [name.decode('utf-8') for name in meta[b'label_names']]

# Display some images
fig, axes = plt.subplots(2, 5, figsize=(10, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(X_train[i])
    ax.set_title(class_names[np.argmax(y_train[i])])
    ax.axis('off')

plt.show()

[2025-03-04 15:00:48,376: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-04 15:00:48,386: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-04 15:00:48,390: INFO: common: created directory at: artifacts]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz', 'local_data_file': 'artifacts/data_ingestion/data.tar.gz', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}, 'training': {'root_dir': 'artifacts/training', 'trained_model_path': 'artifacts/training/model.h5', 'test_data': 'artifacts/data_ingestion/cifar-10-batches-py/test_batch', 'train_data': 'artifacts/data_ingestion/cifar-10-batches-py'}}


BoxKeyError: "'ConfigBox' object has no attribute 'train_data'"

'artifacts\\data_ingestion\\cifar-10-batches-py\\data_batch_1'